#### 다음 실습 코드는 학습 목적으로만 사용 바랍니다. 문의 : audit@korea.ac.kr 임성열 Ph.D.

In [ ]:
# 최신 crewAI가 지원하는 버전 조합으로 구성
# crewAI는 내부적으로 langchain 및 openai, crewai_tools 등과 강하게 연동되어 필요 시 하위 호환 문제 조치
# 예) openai==1.55.3는 최근 릴리스로 구버전의 crewAI와 충돌할 수 있어, 안정적으로 동작하는 1.25.1을 사용
# 별도 가상환경 생성 후, 실행 권고 (특정 버전을 맞춘 상태임)

!pip install crewai==0.28.8 \
            crewai_tools==0.1.6 \
            langchain==0.1.14 \
            langchain_community==0.0.30 \
            openai==1.25.1 \
            httpx==0.27.0 \
            python-dotenv

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [ ]:
import os

# .env 파일 불러오기
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# 키가 없을 경우 예외 발생
if not api_key:
    raise ValueError("API 키가 비어 있습니다. 올바른 키를 입력하세요.")

# 환경 변수로 설정
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"


Agent(에이전트) 생성하기

Agent를 정의하고, role(역할), goal(목표), backstory(배경 설명)를 제공합니다.

LLM(대규모 언어 모델)은 롤플레잉을 할 때 더 나은 성능을 보이는 것으로 확인되었습니다.

### Agent: 기획자(Planner)

**참고**: _여러 문자열_ 사용의 이점:
```Python
varname = "텍스트의 첫 번째 줄"
          "텍스트의 두 번째 줄"
```

_삼중 따옴표 독스트링_ 사용과 비교했을 때:
```Python
varname = """텍스트의 첫 번째 줄
             텍스트의 두 번째 줄
          """
```
여러 문자열을 사용하면 공백과 줄바꿈 문자가 추가되는 것을 피할 수 있어, LLM에 전달할 때 더 나은 형식을 유지할 수 있습니다.

In [6]:
from crewai import Task

task_outputs = {}  # 전역 저장소

class TrackingTask(Task):
    def execute(self, context=None):
        result = super().execute(context=context)
        # name이 존재하면 결과 저장
        key = getattr(self, "name", None)  # pydantic에서는 선언된 필드여야 함
        if key:
            task_outputs[key] = result
        return result


In [7]:
# 콘텐츠 기획자
planner = Agent(
    role="콘텐츠 기획자",
    goal="{topic}에 대한 흥미롭고 사실에 기반한 콘텐츠 기획",
    backstory="당신은 {topic}에 대한 블로그 글을 "
              "기획하는 일을 하고 있습니다. "
              "독자들이 새로운 것을 배우고 "
              "정보에 기반한 결정을 내릴 수 있도록 "
              "도움이 되는 정보를 수집합니다. "
              "당신의 작업은 콘텐츠 작성자가 "
              "이 주제로 글을 쓸 수 있는 토대가 됩니다. "
              "결과물은 한국어로 생성될 예정입니다.",
    allow_delegation=False,
    verbose=True
)

### Agent: Writer

In [8]:
writer = Agent(
    role="콘텐츠 작성자",
    goal="{topic}에 대한 통찰력 있고 사실에 기반한 "
         "의견 기사 작성",
    backstory="당신은 {topic}에 대한 새로운 "
              "의견 기사를 작성하고 있습니다. "
              "콘텐츠 기획자가 제공한 개요와 "
              "주제 관련 맥락을 바탕으로 글을 씁니다. "
              "콘텐츠 기획자가 제시한 "
              "주요 목표와 방향을 따릅니다. "
              "또한 객관적이고 공정한 통찰을 제공하고 "
              "이를 콘텐츠 기획자가 제공한 "
              "정보로 뒷받침합니다. "
              "의견 기사에서 객관적 진술과 "
              "구별되는 개인적 의견을 명시합니다."
              "결과물은 한국어로 생성될 예정입니다.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [9]:
editor = Agent(
    role="편집자",
    goal="주어진 블로그 글을 조직의 글쓰기 스타일에 "
         "맞게 편집",
    backstory="당신은 콘텐츠 작성자로부터 "
              "블로그 글을 받는 편집자입니다. "
              "블로그 글을 검토하여 저널리즘의 모범 사례를 "
              "따르고 있는지 확인하고, "
              "의견이나 주장을 제시할 때 "
              "균형 잡힌 관점을 제공하며, "
              "가능한 한 주요 논쟁적 주제나 "
              "의견을 피하도록 하는 것이 당신의 목표입니다."
              "결과물은 한국어로 생성될 예정입니다.",
    allow_delegation=False,
    verbose=True
)

## Task(작업) 생성하기

- Task를 정의하고, `description`(설명), `expected_output`(예상 결과물), `agent`(수행 에이전트)를 제공합니다.


### Task: Plan

In [10]:
plan = TrackingTask(
    track_name="planner",   # 결과 저장용 키
    name="planner",         # 내부 작업용 이름
    description=(
        "1. {topic}에 대한 최신 트렌드, 주요 인물, "
        "그리고 주목할 만한 뉴스를 우선순위화하기.\n"
        "2. 목표 독자층의 관심사와 pain point를 "
        "고려하여 파악하기.\n"
        "3. 서론, 핵심 요점, 행동 유도(CTA)를 포함한 "
        "상세한 콘텐츠 개요 개발하기.\n"
        "4. SEO 키워드와 관련 데이터 또는 출처 포함하기."
    ),
    expected_output=(
        "개요, 독자층 분석, "
        "SEO 키워드, 참고 자료를 포함한 "
        "포괄적인 콘텐츠 기획 문서. "
        "결과물은 한국어로 생성될 예정입니다."
    ),
    agent=planner
)


### Task: Write

In [11]:
write = TrackingTask(
    name="writer",          # Crew 내부에서 사용하는 작업 이름
    track_name="writer",    # 결과 저장용 키
    description=(
        "1. 콘텐츠 기획을 활용하여 {topic}에 대한 "
        "매력적인 블로그 글 작성하기.\n"
        "2. SEO 키워드를 자연스럽게 포함하기.\n"
        "3. 섹션/부제목을 흥미롭게 적절한 "
        "이름으로 지정하기.\n"
        "4. 매력적인 서론, 통찰력 있는 본문, "
        "요약하는 결론으로 글을 구성하기.\n"
        "5. 문법 오류를 점검하고 "
        "브랜드의 톤앤매너와 일치하는지 확인하기.\n"
    ),
    expected_output=(
        "각 섹션이 2~3개의 문단으로 구성된, "
        "출판 준비가 완료된 마크다운 형식의 "
        "잘 작성된 블로그 글. "
        "결과물은 한국어로 생성될 예정입니다."
    ),
    agent=writer
)

### Task: Edit

In [12]:
edit = TrackingTask(
    name="editor",           # 내부 Task 이름
    track_name="editor",     # 결과 저장 키
    description=(
        "주어진 블로그 글의 문법 오류를 검토하고 "
        "브랜드의 톤앤매너와 일치하는지 확인하되, "
        "**검토 완료 또는 상태 보고 문장은 출력하지 말고**, "
        "**수정된 블로그 글만 출력할 것.**"
    ),
    expected_output=(
        "각 섹션이 2~3개의 문단으로 구성된, "
        "출판 준비가 완료된 마크다운 형식의 "
        "잘 작성된 블로그 글. "
        "**단, 검토 상태를 알리는 문장은 포함하지 말고, 글 내용만 출력할 것.** "
        "결과물은 한국어로 생성됩니다."
    ),
    agent=editor
)

## Crew(팀) 생성하기

- Agent들로 구성된 팀을 생성합니다
- 해당 Agent들이 수행할 작업들을 전달합니다.
    - **참고**: *이 간단한 예시에서는* 작업들이 순차적으로 수행됩니다(즉, 서로 의존적임). 따라서 목록에서의 작업 _순서_가 _중요_합니다.
- `verbose=2`를 설정하면 실행의 모든 로그를 확인할 수 있습니다.


이 설정에서:
1. `planner`가 먼저 콘텐츠를 기획합니다
2. `writer`가 기획된 내용을 바탕으로 글을 작성합니다
3. `editor`가 최종적으로 작성된 글을 검토하고 편집합니다

In [13]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

In [14]:
# 크루 실행
topic ="LLM(Large Language Model)을 이용한 지능형 에이전트 경쟁력 제고 방안"
answer = crew.kickoff(inputs={"topic": topic})


 [DEBUG]: == Working Agent: 콘텐츠 기획자
 [INFO]: == Starting Task: 1. LLM(Large Language Model)을 이용한 지능형 에이전트 경쟁력 제고 방안에 대한 최신 트렌드, 주요 인물, 그리고 주목할 만한 뉴스를 우선순위화하기.
2. 목표 독자층의 관심사와 pain point를 고려하여 파악하기.
3. 서론, 핵심 요점, 행동 유도(CTA)를 포함한 상세한 콘텐츠 개요 개발하기.
4. SEO 키워드와 관련 데이터 또는 출처 포함하기.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.

---

**LLM(Large Language Model)을 이용한 지능형 에이전트 경쟁력 제고 방안 콘텐츠 기획**

**1. 개요**

- **서론**: 
   - LLM(Large Language Model)의 발전과 지능형 에이전트의 역할 변화
   - LLM을 통한 지능형 에이전트의 경쟁력 강화 필요성
   - 이 글의 목적: LLM 활용 방안과 최신 트렌드를 통해 독자에게 정보 제공

- **핵심 요점**:
  1. **LLM을 이용한 지능형 에이전트 최신 트렌드**
     - NLP(자연어 처리)의 진보와 LLM의 역할
     - AI 기반 고객 서비스 및 챗봇의 혁신
     - 개인화된 사용자 경험 제공
     - 데이터 보안 및 윤리적 문제 고려

  2. **주요 인물 및 기업**
     - OpenAI, Google, Microsoft 등 LLM 연구 및 적용 선두 기업
     - 주요 AI 연구자와 개발자들의 기여

  3. **주목할 만한 뉴스 및 사례**
     - GPT-4, BERT 등 최신 모델의 출시 및 적용 사례
     - 기업의 성공적인 LLM 적용 사례 (예: AI 기반 고객 

In [32]:
from IPython.display import Markdown
Markdown(answer)

```markdown
# LLM(Large Language Model)을 활용한 지능형 에이전트 경쟁력 제고 방안

## LLM의 발전과 지능형 에이전트의 혁신

LLM(Large Language Model)과 지능형 에이전트의 발전은 기술 산업 전반에 걸쳐 혁신을 이끌고 있습니다. 이 블로그 글에서는 LLM을 활용하여 지능형 에이전트의 경쟁력을 높이는 방법에 대해 알아봅니다. 최신 트렌드, 주요 인물, 그리고 주목할 만한 뉴스를 통해 독자들은 이 분야의 미래를 조망할 수 있을 것입니다.

## LLM과 지능형 에이전트의 최신 트렌드

최근 LLM의 발전은 자연어 처리(NLP) 분야에서 눈부신 기술적 향상을 이루었습니다. 이러한 발전은 지능형 에이전트의 능력을 크게 증대시켜 다양한 실제 응용 사례를 가능하게 하고 있습니다. 예를 들어, 고객 서비스 챗봇은 더욱 자연스러운 대화를 통해 사용자 경험을 향상시키고 있습니다. 이는 개인화된 서비스를 제공하는 데 중요한 역할을 하고 있으며, 소비자의 요구를 보다 정확히 파악할 수 있게 합니다.

## 주요 인물과 그들의 기여

OpenAI와 Google AI의 연구자 및 리더들은 LLM 기술의 선두주자로 자리매김하고 있습니다. 이들 기업의 연구자들은 LLM의 성능을 지속적으로 개선하고 있으며, 학계와 산업계에서도 주목받고 있습니다. 특히, LLM과 NLP 분야에서 활약 중인 여러 연구자들은 새로운 알고리즘 개발과 성능 최적화에 중점을 두고 있습니다.

## 주목할 만한 뉴스와 적용 사례

최근 LLM 관련 기술 발표는 지능형 에이전트의 새로운 적용 사례를 제시하고 있습니다. 예를 들어, 의료 분야에서는 LLM을 활용한 진단 보조 시스템이 개발되어 의료진의 업무 효율을 크게 높이고 있습니다. 이러한 사례들은 LLM 기술이 다양한 산업에서 어떻게 활용될 수 있는지를 보여주며, 앞으로의 가능성을 시사합니다.

## 경쟁력 제고를 위한 전략

경쟁력을 높이기 위해서는 사용자 경험 개선을 위한 개인화 전략이 필수적입니다. 이를 위해서는 효율적인 알고리즘을 활용하여 성능을 최적화해야 합니다. 또한, 지속적인 모델 업데이트와 사용자 피드백을 통한 개선 과정이 필요합니다. 이러한 전략을 통해 기업은 LLM을 활용한 지능형 에이전트의 경쟁력을 강화하고, 시장에서의 입지를 확고히 할 수 있습니다.

## 결론

LLM을 활용한 지능형 에이전트는 다양한 산업에서 혁신을 주도하고 있습니다. 최신 기술을 통해 경쟁력을 확보하고, 성장의 기회를 놓치지 마세요. 이제 여러분의 비즈니스에 LLM을 활용해 보세요. 지능형 에이전트의 발전은 계속될 것이며, 이를 통해 새로운 비즈니스 기회를 창출할 수 있습니다.

이 글을 통해 독자들은 LLM 기술의 최신 동향을 파악하고, 실질적인 비즈니스 적용 방안을 모색할 수 있을 것입니다. 기술 산업에 종사하는 이들에게는 특히 더 유용한 인사이트가 될 것입니다.
```